In [1]:
import pickle

with open('sample_generator_config.pickle', 'rb') as f:
    config = pickle.load(f)

In [2]:
config.keys()

dict_keys(['var_num', 'sample_size', 'num_interactions', 'error_scale', '_main_beta', 'interactions_coef', 'beta_effective', '_interactions', '_main_coefficients'])

In [3]:
config['_interactions']

{x_1*x_2: 6.5235939169685775,
 x_1*x_3: -7.200809560873655,
 x_1*x_4: -7.8066647172659325,
 x_2*x_3: 7.280589248544736,
 x_2*x_4: -6.157619565739074,
 x_3*x_5: -5.435590601140175,
 x_4*x_5: -5.256113102962207,
 x_1*x_2*x_4: -7.05279343734103,
 x_1*x_3*x_4: -7.213887640336632,
 x_1*x_3*x_5: -6.654268815787151,
 x_2*x_3*x_4: -7.424751777505572,
 x_2*x_4*x_5: 6.977620402014335,
 x_3*x_4*x_5: -4.501430112603433,
 x_1*x_2*x_3*x_4: 6.648499802383317,
 x_1*x_2*x_3*x_5: 6.675623047942929,
 x_1*x_3*x_4*x_5: -5.097119184879741,
 x_2*x_3*x_4*x_5: -4.976458063791224}

In [22]:
from importance_score import dask_parameter_generator
from dask import dataframe as dd

In [23]:
ddf = dd.read_parquet('sample_data.parquet')

In [24]:
original_y = ddf.y.copy()

In [25]:
from scipy.stats import norm
def add_error(y, scale):
    length = len(y)
    return y + norm.rvs(scale = scale, size = length)


In [26]:
ddf['y'] = ddf.y.map_partitions(add_error, 3)

In [27]:
ddf['y'].compute()

0      10.144614
1      20.469815
2      -7.328705
3      11.779278
4      -3.786160
         ...    
932     1.801861
933    11.002508
934    -0.043215
935    14.690041
936    -8.170054
Name: y, Length: 15000, dtype: float64

In [28]:
original_y.compute()

0      13.090290
1      17.349518
2      -0.405313
3      11.214214
4       2.817946
         ...    
932    -0.260962
933     7.234470
934     5.335092
935    11.385201
936    -9.787877
Name: y, Length: 15000, dtype: float64

In [17]:
ddf['y'] = ddf.y.map_partitions(add_error, 3).compute()

ValueError: cannot reindex on an axis with duplicate labels

In [19]:
ddf_sample = ddf.sample(frac = .25)

In [31]:
ddf_sample

,x_1,x_2,x_3,x_4,x_5,y,barcode
npartitions=17,,,,,,,
,int64,int64,int64,int64,int64,float64,uint8
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [21]:
gen = dask_parameter_generator(ddf_sample)

In [22]:
gen.gen_importance_score(max_variables = 5)

100%|██████████| 83681/83681 [03:13<00:00, 432.20it/s]


In [23]:
import pandas as pd

df = pd.read_csv('importance_score.csv')

In [24]:
columns = df.columns[df.columns.str.contains(',')].tolist() + ['score']
df = df.loc[:, columns].sort_values(['score'], ascending = False).reset_index(drop = True)
df


,"beta_1,2","beta_1,3","beta_1,4","beta_1,5","beta_2,3","beta_2,4","beta_2,5","beta_3,4","beta_3,5","beta_4,5",...,"beta_2,3,5","beta_2,4,5","beta_3,4,5","beta_1,2,3,4","beta_1,2,3,5","beta_1,2,4,5","beta_1,3,4,5","beta_2,3,4,5","beta_1,2,3,4,5",score
0,0,0,0,1,0,1,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1.000000
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,1,0,1.000000
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,1,0,1.000000
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,1.000000
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83676,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0.050149
83677,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.045482
83678,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.043544
83679,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0.022128


In [25]:
import numpy as np
df = df.loc[df.score.gt(0), :].reset_index(drop = True)
result = df.apply(np.average, weights = df.score, axis = 0).sort_values(ascending = False)
result.pop('score')
ranking = pd.DataFrame(zip(result.index.tolist(), result), columns = ['coefficients', 'score'])
ranking['ranking'] = ranking.score.rank(ascending = False, method = 'min')


In [44]:
from sympy import Symbol
interactions = {str(key):val for key, val in config['_interactions'].items()}

{'x_1*x_2': 6.5235939169685775,
 'x_1*x_3': -7.200809560873655,
 'x_1*x_4': -7.8066647172659325,
 'x_2*x_3': 7.280589248544736,
 'x_2*x_4': -6.157619565739074,
 'x_3*x_5': -5.435590601140175,
 'x_4*x_5': -5.256113102962207,
 'x_1*x_2*x_4': -7.05279343734103,
 'x_1*x_3*x_4': -7.213887640336632,
 'x_1*x_3*x_5': -6.654268815787151,
 'x_2*x_3*x_4': -7.424751777505572,
 'x_2*x_4*x_5': 6.977620402014335,
 'x_3*x_4*x_5': -4.501430112603433,
 'x_1*x_2*x_3*x_4': 6.648499802383317,
 'x_1*x_2*x_3*x_5': 6.675623047942929,
 'x_1*x_3*x_4*x_5': -5.097119184879741,
 'x_2*x_3*x_4*x_5': -4.976458063791224}

In [41]:
Symbol('x_{1} * x_2')

x_{1} * x_2

In [27]:
ranking

,coefficients,score,ranking
0,"beta_3,5",0.182749,1.0
1,"beta_2,4,5",0.182749,1.0
2,"beta_1,3,5",0.182749,1.0
3,"beta_1,3",0.182749,1.0
4,"beta_4,5",0.182749,1.0
5,"beta_1,3,4",0.182749,1.0
6,"beta_2,3",0.182749,1.0
7,"beta_2,4",0.182749,1.0
8,"beta_1,4",0.182749,1.0
9,"beta_1,2",0.182749,10.0


In [17]:
config

{'var_num': 4,
 'sample_size': 10000,
 'num_interactions': 8,
 'error_scale': 2.23606797749979,
 '_main_beta': [beta_1, beta_2, beta_3, beta_4],
 'interactions_coef': [4.297348381298434,
  7.579078200375563,
  4.51452875165397,
  7.773754398278939,
  -3.0526023172560035,
  6.462084713309809,
  5.938332175329972,
  -4.163256468935406],
 'beta_effective': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1],
 '_interactions': {x_1*x_2: 4.297348381298434,
  x_1*x_3: 7.579078200375563,
  x_1*x_4: 4.51452875165397,
  x_2*x_3: 7.773754398278939,
  x_1*x_2*x_3: -3.0526023172560035,
  x_1*x_3*x_4: 6.462084713309809,
  x_2*x_3*x_4: 5.938332175329972,
  x_1*x_2*x_3*x_4: -4.163256468935406},
 '_main_coefficients': [6.536201617172051,
  4.677226350482356,
  7.7569712979465475,
  -3.2466757680253466,
  6.843976305009161]}

In [6]:
cell_means = gen.find_groupby_means().compute().y.to_numpy()
kwargs = {"L_inv": gen.L_inv, "cell_means": cell_means, "cell_means_covariance": gen.cell_means_covariance, "sample_size": gen.sample_size}


In [5]:
contrasts = gen.contrast_generator()

In [8]:
cell_means_contrasts = next(contrasts).toarray()
cell_means_contrasts

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=uint8)

In [8]:
cell_means = gen.find_groupby_means().compute().y.to_numpy()
L_inv = gen.L_inv
cell_means_covariance = gen.cell_means_covariance; sample_size = gen.sample_size
contrast = cell_means_contrasts

In [9]:
nu_1 = contrast.shape[0]
nu_2 = sample_size - L_inv.shape[0]

LC = contrast @ L_inv.T
mu = LC @ cell_means
var = LC @ cell_means_covariance @ LC.T


In [10]:
from numpy.linalg import inv
mu.T @ inv(var) @ mu

10.9320176219622

In [15]:
mu

array([-1.63158809])

In [15]:
import numpy as np
cell_means_contrasts.sum(axis = 0).astype(np.uint8)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=uint8)

In [8]:
result = gen.partial_f_test(cell_means_contrasts, gen.L_inv ,gen.find_groupby_means().compute().y.to_numpy(), gen.cell_means_covariance, gen.sample_size)

In [11]:
result.p_value

1.7192002638196008e-05

In [14]:
gen.cell_means_covariance

<16x16 sparse matrix of type '<class 'numpy.float64'>'
	with 16 stored elements (1 diagonals) in DIAgonal format>

In [58]:
import scipy
import numpy as np
all_interaction_betas = scipy.sparse.eye(gen.num_full_var - gen.num_main_var -1, gen.num_full_var, k = gen.num_main_var + 1).astype(np.uint8).toarray()
all_interaction_betas

array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=uint8)

In [7]:
cell_means_contrasts = next(contrasts).toarray()
cell_means_contrasts

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=uint8)

In [36]:
c = cell_means_contrasts[0,:]
c

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], dtype=uint8)

In [68]:
from scipy.sparse.linalg import inv as lin_inv
from numpy.linalg import inv
import numpy as np
from scipy.stats import f
L_inv = lin_inv(gen.L)
L_inv = L_inv.astype(np.int8)
group_means = gen.find_groupby_means().compute().y.to_numpy()
cell_means_var = gen.cell_means_covariance

In [34]:
all_interaction_betas

array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=uint8)

In [97]:
cell_means_var

<16x16 sparse matrix of type '<class 'numpy.float64'>'
	with 16 stored elements (1 diagonals) in DIAgonal format>

In [95]:
from numpy.linalg import eig
P = eig(gen.L.toarray())[0]

In [96]:
P

array([ 1.        +0.j        ,  2.10039273+0.j        ,
       -0.79443771+0.54592453j, -0.79443771-0.54592453j,
       -0.19211522+0.96054922j, -0.19211522-0.96054922j,
        0.43635657+0.58615956j,  0.43635657-0.58615956j,
        1.66040772+0.j        ,  0.3619187 +0.75706719j,
        0.3619187 -0.75706719j, -0.24490725+0.94672802j,
       -0.24490725-0.94672802j, -1.        +0.j        ,
       -0.89443062+0.j        ,  1.        +0.j        ])

In [73]:
# with c
LC = c@L_inv.T
mu = LC.dot(group_means)
var = LC@cell_means_var@LC.T
# scaler
f_value = mu*mu/var/c.shape[0]
# f_value = mu.dot(inv(var)).dot(mu.T)/c.shape[0]
f_value
cdf = f.cdf(f_value, c.shape[0], gen.sample_size-gen.num_full_var, loc=0, scale=1)

In [74]:
cdf

0.9999999999999999

In [41]:
LC = all_interaction_betas@L_inv.T
mu = LC.dot(group_means)
all_var = LC@cell_means_var@LC.T


In [57]:
LC = cell_means_contrasts@L_inv.T
mu = LC.dot(group_means)
var = LC@cell_means_var@LC.T
var

array([[ 0.1286896 ,  0.10081372, -0.1286896 ],
       [ 0.10081372,  0.19688436, -0.19688436],
       [-0.1286896 , -0.19688436,  0.2435122 ]])

In [56]:
cell_means_contrasts[:, 5:] @ all_var @cell_means_contrasts[:, 5:].T

array([[ 0.1286896 ,  0.10081372, -0.1286896 ],
       [ 0.10081372,  0.19688436, -0.19688436],
       [-0.1286896 , -0.19688436,  0.2435122 ]])

In [62]:
gen.L.shape

(16, 16)

In [64]:
gen.cell_means_covariance

<16x16 sparse matrix of type '<class 'numpy.float64'>'
	with 16 stored elements (1 diagonals) in DIAgonal format>

In [ ]:
        c = beta_contrasts #self.gen_beta_contrasts(beta)
        assert c.shape[0] <= c.shape[1]
        LC = L_inv.dot(c.T)
        mu = LC.T.dot(group_means) #self.cluster_estimator.full_to_reduced_with_counts.y
        var = LC.T.dot(full_var).dot(LC)
        f_value = mu.T.dot(inv(var)).dot(mu)/c.shape[0]
        cdf = f.cdf(f_value, c.shape[0], N-p, loc=0, scale=1)
        return cdf    


In [33]:
from numpy.linalg import svd, qr

In [34]:
svd(contrasts)

(array([[-0.70710678, -0.70710678],
        [-0.70710678,  0.70710678]]),
 array([1.30656296, 0.5411961 ]),
 array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         -0.92387953,  0.        ,  0.38268343],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         -0.38268343,  0.        , -0.92387953],
        [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
          0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
          0.        ,  0.        ,  0.        ],
        [-1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  1.        ,  0.        ],
        [ 0.        ,  1.      

In [51]:
svd(contrasts.T @ inv(contrasts @ contrasts.T) @ contrasts)[-1].round(3)

array([[-0., -0., -0., -0., -0., -1., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [ ]:
        if isinstance(beta, list):
            beta = np.array(beta)
        beta = beta.reshape(-1)
        assert beta.shape[0] == 2**self.sample_generator.p
        assert beta.max() == 1
        where_ones= np.where(beta)[0]
        if where_ones.shape[0] == 1:
            return beta.reshape(1, -1)
        else:
            N = 2**self.sample_generator.p
            contrasts = []
            for idx, beta_idx in enumerate(where_ones):
                if idx:
                    c_compare = np.zeros(N, dtype = np.byte).reshape(1, -1); c_compare[0,beta_idx] = 1
                    c = c_base - c_compare
                    contrasts.append(c/norm(c, 2))
                    c_base = c.copy(); c_base[0, c_base_index ] += 1
                else:
                    c_base = np.zeros(N, dtype = np.byte).reshape(1, -1)
                    c_base[0,beta_idx] = 1
                    contrasts.append(c_base)
                    c_base_index = beta_idx
            contrasts = np.concatenate(contrasts, axis = 0)
            return contrasts

In [ ]:
    @staticmethod
    def get_cdf(beta_contrasts, L_inv, full_var, group_means, N, p):
        c = beta_contrasts #self.gen_beta_contrasts(beta)
        assert c.shape[0] <= c.shape[1]
        LC = L_inv.dot(c.T)
        mu = LC.T.dot(group_means) #self.cluster_estimator.full_to_reduced_with_counts.y
        var = LC.T.dot(full_var).dot(LC)
        f_value = mu.T.dot(inv(var)).dot(mu)/c.shape[0]
        cdf = f.cdf(f_value, c.shape[0], N-p, loc=0, scale=1)
        return cdf    


    @property
    def all_available_beta_for_test(self):
        if hasattr(self, '_all_available_beta_for_test'):
            pass
        else:
            non_zero_part = product([0,1], repeat = 2**self.sample_generator.p - self.sample_generator.p - 1)
            zeros = [0] + [0 for _ in range(self.sample_generator.p)]
            if hasattr(self, '_beta_sum_range'):
                beta_range = list(range(self.beta_sum_range[0], self.beta_sum_range[1]+1))
                self._all_available_beta_for_test = np.array([zeros + list(x) for x in non_zero_part if sum(x) in beta_range ]) #if sum(x)
            else:
                self._all_available_beta_for_test = np.array([zeros + list(x) for x in non_zero_part if sum(x)]) #if sum(x)
        return self._all_available_beta_for_test


    def gen_beta_contrasts(self, beta):
        if isinstance(beta, list):
            beta = np.array(beta)
        beta = beta.reshape(-1)
        assert beta.shape[0] == 2**self.sample_generator.p
        assert beta.max() == 1
        where_ones= np.where(beta)[0]
        if where_ones.shape[0] == 1:
            return beta.reshape(1, -1)
        else:
            N = 2**self.sample_generator.p
            contrasts = []
            for idx, beta_idx in enumerate(where_ones):
                if idx:
                    c_compare = np.zeros(N, dtype = np.byte).reshape(1, -1); c_compare[0,beta_idx] = 1
                    c = c_base - c_compare
                    contrasts.append(c/norm(c, 2))
                    c_base = c.copy(); c_base[0, c_base_index ] += 1
                else:
                    c_base = np.zeros(N, dtype = np.byte).reshape(1, -1)
                    c_base[0,beta_idx] = 1
                    contrasts.append(c_base)
                    c_base_index = beta_idx
            contrasts = np.concatenate(contrasts, axis = 0)
            return contrasts

In [1]:
from dataclasses import dataclass
from typing import  Union, Tuple
import numpy as np
from numpy.linalg import inv
from scipy.stats import bernoulli, norm
import pandas as pd
from itertools import combinations, product
from math import comb, sqrt
from sympy import Symbol
from tempfile import NamedTemporaryFile
from csv import DictWriter, writer
import os

In [11]:
for i in zip(range(10), range(10, 20)):
    print(i)

(0, 10)
(1, 11)
(2, 12)
(3, 13)
(4, 14)
(5, 15)
(6, 16)
(7, 17)
(8, 18)
(9, 19)


In [26]:
from random_sample_generator import sample_generator
import dask
from dask import dataframe as dd
from scipy.stats import bernoulli, norm
import numpy as np
self = sample_generator(10, 1000)

In [2]:
from dask.distributed import Client, progress
client = Client()

In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 15.55 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44021,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 15.55 GiB
Comm: tcp://127.0.0.1:34191,Total threads: 4
Dashboard: http://127.0.0.1:34039/status,Memory: 3.89 GiB
Nanny: tcp://127.0.0.1:33757,


In [16]:
import dask.bag as db

self = sample_generator(10, 1_000_000)

b = db.range(self.sample_size, npartitions = self.sample_size//1000)
output = db.map(lambda x: bernoulli.rvs(list(self.pi.values()), size = self.var_num).tolist(), b).compute()

In [20]:
import pandas as pd

pd.DataFrame(output[:10], columns = [str(x) for x in self.fieldnames][:-1])
pd.DataFrame(output[:10], columns = self.fieldnames[:-1])

,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10
0,1,0,1,0,1,0,0,1,0,1
1,0,0,0,1,1,0,1,0,1,0
2,1,0,1,1,1,1,0,1,0,1
3,0,1,1,1,1,0,0,0,0,1
4,1,0,0,1,1,1,0,0,0,1
5,1,0,0,0,1,0,0,0,1,0
6,0,0,0,1,0,0,0,0,0,0
7,0,0,0,0,1,0,0,1,0,1
8,1,0,0,1,1,1,0,0,0,1
9,1,0,0,1,1,0,0,1,1,0


In [22]:


ddf = dd.from_pandas(pd.DataFrame(output, columns = self.fieldnames[:-1]), npartitions = self.sample_size//1000)

In [23]:
ddf

,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10
npartitions=1000,,,,,,,,,,
0,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
1000,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
999000,...,...,...,...,...,...,...,...,...,...
999999,...,...,...,...,...,...,...,...,...,...


In [29]:
ddf['y'] = ddf.apply(get_y, axis = 1, meta = (None, 'float64'))

In [31]:
ddf.to_csv('test.csv', index = False)

/root/barcode/.venv/lib/python3.10/site-packages/distributed/client.py:3108: UserWarning: Sending large graph of size 76.81 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


KeyboardInterrupt: 

In [24]:
def get_y(x):
    main_effect_variables = self.main_effect_variables_str
    main_effect_coefficients = self.main_coefficients
    interaction_effect_coefficients = self.interaction_effect_coefficients
    main_effect = sum([coef*variable for coef, variable in zip(main_effect_coefficients.values(), np.concatenate([[1], x]))])

    x_row = pd.Series(x, index = main_effect_variables)
    interaction_effect = sum([x_row[str(key).split('*')].prod() * val for key, val in self.interaction_effect_coefficients.items()])
    error = norm.rvs(scale = self.error_scale)
    y = main_effect + interaction_effect + error
    return y




In [32]:
self.interactions_coef

[-5.8685432877824955,
 6.542922321060808,
 5.067777842957762,
 5.8185041758517375,
 -7.702982451809929,
 5.780085655546664,
 3.565711510076809,
 5.412489901527204,
 3.8018096791149123,
 5.804481021662618,
 5.541817576859012,
 7.925822874109593,
 -7.056561407518027,
 -5.3154959796462204,
 7.964252030583758,
 3.7868148893960076,
 -4.152223007230901,
 7.502640685042922,
 4.077402702550701,
 -3.2995730545818374,
 -6.324137233782361,
 4.388740981808253,
 7.05915032026471,
 4.568458743339698,
 -6.961494527786105,
 7.614740826143036,
 3.19902380860176,
 -6.261623609721586,
 -3.1844819242856097,
 7.246639820428283,
 4.444860104191826,
 6.974200253615788,
 -3.187346781845021,
 -3.307129972533024,
 -4.198446072304836,
 5.059059620553298,
 -5.7351831175283685,
 4.071821149116886,
 -3.60546136319537,
 -4.937010380694728,
 3.107097800531913,
 -6.57054204556367,
 3.4349198500221445,
 -7.04490001364966,
 -4.523612558307742,
 5.488299072309177,
 6.95595546655528,
 6.140838281127812,
 6.672610944926928

In [ ]:
    @property
    def y(self):
        if hasattr(self, '_y'):
            pass
        else:
            beta = self.beta
            interactions = self.interactions
            y = []
            
            def find_interaction_effect(row, interactions = self.interactions):
                output = 0
                for key, val in interactions.items():
                    output += row[key.split('*')].prod() * val
                return output
            
            
            for idx, row in self.X.iterrows():
                main = beta['beta_0'] + sum([x*y for x,y in zip(list(beta.values())[1:], row)])
                interaction_eff = find_interaction_effect(row)
                error = norm.rvs(scale = self.error_scale)
                y.append(main + interaction_eff + error)
            
            self._y = np.array(y).reshape(-1,1)
        return self._y


In [34]:
from sympy import Symbol

x_generator = self.X_generator()
variables = self.main_effect_variables + [Symbol('y')]
def return_dict(array_x):
    y = self.get_y(array_x)
    return {var_name: [val] for var_name, val in zip(variables, array_x.tolist() + [y])}
# outputs = []
# for array_x in x_generator:
#     y = client.submit(self.get_y, array_x)
#     output = client.submit(return_dict, array_x, y)
#     outputs.append(output)

In [35]:
# import pandas as pd
# df = client.submit(pd.DataFrame, outputs)


In [36]:
from dask import bag

b = bag.from_sequence(x_generator)
output = b.map(return_dict).sum().compute()

2023-05-22 23:47:30,999 - distributed.worker - WARNING - Compute Failed
Key:       ('return_dict-sum-part-3af5277e4e00744f2bbf523414cdc896', 8)
Function:  execute_task
args:      ((<function empty_safe_apply at 0x7f69a8bc4790>, <built-in function sum>, (<function map_chunk at 0x7f69a8bc40d0>, <function return_dict at 0x7f6992182dd0>, [[array([0, 0, 0, 0, 0, 1, 1, 1, 0, 1]), array([1, 1, 1, 1, 0, 1, 1, 0, 0, 0]), array([0, 1, 0, 1, 0, 1, 1, 1, 1, 0]), array([0, 0, 0, 0, 1, 1, 0, 0, 1, 0]), array([1, 0, 1, 0, 0, 1, 1, 0, 0, 1]), array([1, 0, 0, 0, 0, 1, 0, 1, 0, 0]), array([1, 0, 1, 0, 1, 0, 1, 1, 0, 0]), array([1, 0, 1, 0, 1, 1, 1, 0, 0, 1]), array([1, 0, 0, 0, 0, 0, 0, 0, 1, 0]), array([1, 1, 0, 0, 0, 0, 0, 1, 0, 1])]], None, {}), False))
kwargs:    {}
Exception: 'TypeError("unsupported operand type(s) for +: \'int\' and \'dict\'")'

2023-05-22 23:47:31,000 - distributed.worker - WARNING - Compute Failed
Key:       ('return_dict-sum-part-3af5277e4e00744f2bbf523414cdc896', 12)
Function:

TypeError: unsupported operand type(s) for +: 'int' and 'dict'

In [ ]:
output

0

In [24]:
generator = self.generator()
import dask.bag as db
import json

b = db.read_text('data/*.json').map(json.loads)
b


ValueError: An error occurred while calling the from_dict method registered to the pandas backend.
Original Message: If using all scalar values, you must pass an index

In [17]:
self.fieldnames

[x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, y]

In [13]:
dataframes[0].compute()

TypeError: 'tuple' object is not an iterator

In [12]:
outputs = []

for i in range(self.sample_size):
    x_row = dask.delayed(bernoulli.rvs)(list(self.pi.values()), size = self.var_num)
    main_effect = self.get_main_effect_from_x_row(x_row)
    interaction_effect = self.get_interaction_effect_from_x_row(x_row)
    error = dask.delayed(norm.rvs)(scale = self.error_scale)
    y = dask.delayed(sum)([main_effect, interaction_effect, error]).compute()
    outputs.append(x_row.compute().tolist() + [y])


In [14]:
outputs

[[1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 15.192482592032851],
 [1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 15.02561578219487],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 20.418758237435995],
 [0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 17.37662928180334],
 [0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 5.428116418844233],
 [1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 10.541948740656833],
 [1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 24.394359212361632],
 [1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 10.65285969185005],
 [1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 23.93038977106503],
 [0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 12.201882501851838],
 [0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 29.93023868784628],
 [0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 16.31704181905637],
 [1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 14.791099048761684],
 [0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 7.948036352326639],
 [1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 11.664345583834056],
 [1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 8.266578614977707],
 [1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 7.534564276838482],
 [1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 25.90196295838891],
 [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 12.625804307537665],
 [1,

In [37]:
from sklearn.base import BaseEstimator, clone
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics      import r2_score

import numpy as np; import pandas as pd
from typing import List, Union

from bitstring import Bits

In [38]:
!pip install dask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.1 MB/s eta 0:00:0000:0100:010m
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached fsspec-2023.5.0-py3-none-any.whl (160 kB)
  Using cached PyYAML-6.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (682 kB)
  Using cached toolz-0.12.0-py3-none-any.whl (55 kB)


In [42]:
from dask import dataframe as dd

ddf = dd.read_csv('tmpw1zguov5.csv')

In [163]:
def return_bitstring(x_seq):
    b = ''.join((str(z) for z in x_seq))
    bit = Bits(bin = b)
    return bit.uint

In [164]:
ddf['bits'] = ddf.loc[:, ddf.columns.str.contains('x')].apply(return_bitstring, axis = 1)

/root/barcode/.venv/lib/python3.10/site-packages/dask/dataframe/core.py:5917: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'int64'))

  warnings.warn(meta_warning(meta))


In [166]:
result = ddf.set_index('bits').groupby('bits').y.mean()

In [167]:
result.compute()

bits
3     -2.990150
6    -17.113382
7    -12.455653
11   -17.097339
16   -10.500701
18   -18.207829
22   -18.127595
Name: y, dtype: float64

In [104]:
result = ddf.set_index('bits').groupby('bits').y.mean()

In [105]:
result.compute()

bits
3     -2.990150
6    -17.113382
7    -12.455653
11   -17.097339
16   -10.500701
18   -18.207829
22   -18.127595
Name: y, dtype: float64

In [128]:
ddf.set_index('bits')

,x_1,x_2,x_3,x_4,x_5,y
npartitions=1,,,,,,
3,int64,int64,int64,int64,int64,float64
22,...,...,...,...,...,...


In [121]:
ddf

,x_1,x_2,x_3,x_4,x_5,y,bits
npartitions=1,,,,,,,
,int64,int64,int64,int64,int64,float64,int64
,...,...,...,...,...,...,...


In [135]:
ddf.divisions

(None, None)

In [548]:
from itertools import product, combinations
def L(barcode_length):
    def barcode_to_beta(barcode):
        if isinstance(barcode, list):
            output = [1] + barcode
        else:
            output = [1] + list(barcode)
        N = len(output)
        for i in range(2, N):
            output += [np.prod(x) for x in combinations(barcode, i)]
    #     output += [np.prod(output)]
        return output
    all_sets = list(set(product([0,1], repeat = barcode_length))); all_sets.sort()
    L = np.array([barcode_to_beta(x) for x in all_sets]).T
    return L


In [549]:
L(3)

array([[1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 1, 1, 0, 0, 1, 1],
       [0, 1, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1]])

In [1]:
from importance_score import dask_parameter_generator



In [2]:
gen = dask_parameter_generator('tmpw1zguov5.csv')

In [6]:
10 // 3

3

In [565]:
gen.L.toarray()

array([[1, 1, 1, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int8)

In [530]:
gen.MLE_SCALE

0.5345302889099425

In [531]:
gen.find_FIM().toarray().sum()*gen.MLE_VAR

10.000000000000002

In [507]:
ssto, sse = gen.find_error_statistics()

In [513]:
gen.MLE_SCALE

0.5345302889099425

In [533]:
fim = gen.find_FIM()

In [543]:
diags(list(map(lambda x: 1/x if x else float('inf'), fim.diagonal()))).toarray()

array([[inf,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., inf,  0., ...,  0.,  0.,  0.],
       [ 0.,  0., inf, ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ..., inf,  0.,  0.],
       [ 0.,  0.,  0., ...,  0., inf,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., inf]])

In [508]:
gen.MLE_SCALE

In [504]:
gen.sample_size

10

In [494]:
sse.compute()

2.857226297621466

In [455]:
gen.ddf.merge(gen.full_segment_means, how = 'left', left_index = True, right_index = True, suffixes = ['','_pred']).compute()

,y,y_pred
barcode,,
3,-2.990150,-2.990150
6,-17.636538,-17.113382
6,-16.590226,-17.113382
7,-11.383768,-12.455653
7,-13.527537,-12.455653
11,-17.097339,-17.097339
16,-10.500701,-10.500701
18,-18.285188,-18.207829
18,-18.130470,-18.207829


In [421]:
gen.full_segment_means

,barcode,y
0,3,-2.990150
1,6,-17.113382
2,7,-12.455653
3,11,-17.097339
4,16,-10.500701
5,18,-18.207829
6,22,-18.127595


In [419]:
gen

In [420]:
gen.ddf.compute()

KeyError: 7

In [400]:
gen.ddf.index

Dask Index Structure:
npartitions=1
3     uint8
22      ...
Name: barcode, dtype: uint8
Dask Name: sort_index-index, 8 graph layers

In [393]:
gen.ddf

,y
npartitions=1,
3,float64
22,...


In [378]:
gen.full_segment_means.loc[3].y

-17.09733937409042

In [367]:
gen.find_FIM().toarray().sum()

10.0

In [357]:
sdiag = diags(pd.Series(gen.ddf.groupby('barcode').y.count().compute().sort_index()).reindex(list(range(2**5)), fill_value = 0))

In [358]:
type(sdiag)

scipy.sparse._dia.dia_matrix

In [355]:
sdiag.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [352]:
from scipy.sparse import diags


diags()


In [301]:
gen.sample_size

10

In [282]:
gen.gen_basic_parameters()

In [286]:
gen.ddf.shape

(Delayed('int-e2d37237-045f-4e0d-b28c-ecfc07f863f7'), 1)

In [276]:
gen.compute('dask')

,y
npartitions=1,
3,float64
22,...


In [259]:
gen.ddf.compute().to_numpy()

array([[ -2.99015048],
       [-17.63653766],
       [-16.59022621],
       [-11.38376786],
       [-13.52753723],
       [-17.09733937],
       [-10.50070062],
       [-18.28518827],
       [-18.13046978],
       [-18.12759475]])

In [258]:
gen.ddf

,y
npartitions=1,
3,float64
22,...


In [212]:
gen.input_featuers

array([ True,  True,  True,  True,  True, False])

In [204]:
gen.ddf.apply(gen.return_bitstring, axis = 1)

ValueError: Metadata inference failed in `apply`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
TypeError("dask_parameter_generator.return_bitstring() missing 1 required positional argument: 'x_seq'")

Traceback:
---------
  File "/root/barcode/.venv/lib/python3.10/site-packages/dask/dataframe/utils.py", line 193, in raise_on_meta_error
    yield
  File "/root/barcode/.venv/lib/python3.10/site-packages/dask/dataframe/core.py", line 6810, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "/root/barcode/.venv/lib/python3.10/site-packages/dask/utils.py", line 1105, in __call__
    return getattr(__obj, self.method)(*args, **kwargs)
  File "/root/barcode/.venv/lib/python3.10/site-packages/pandas/core/frame

In [192]:
gen.ddf

,x_1,x_2,x_3,x_4,x_5,y
npartitions=1,,,,,,
,int64,int64,int64,int64,int64,float64
,...,...,...,...,...,...


In [189]:
ddf = dd.read_csv('tmpw1zguov5.csv')
ddf

,x_1,x_2,x_3,x_4,x_5,y
npartitions=1,,,,,,
,int64,int64,int64,int64,int64,float64
,...,...,...,...,...,...


In [186]:
gen.ddf

<module 'dask.dataframe' from '/root/barcode/.venv/lib/python3.10/site-packages/dask/dataframe/__init__.py'>

In [118]:
dask.dataframe

TypeError: 'module' object is not callable

In [159]:
from bitstring import BitArray, Bits

x = Bits(bin = "111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111")
y = Bits(bin = "1111111111111111111111111111111111")

In [160]:
type(x.uint)

int

In [162]:
type(x.uintbe)

int